In [6]:
import json
from collections import defaultdict
from sklearn import linear_model, model_selection
import numpy as np
import random
import gzip
import dateutil.parser
import math
import mido
import pretty_midi
import joblib
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec
import collections
import os
import tarfile
from midi_methods import *

%store -r

In [7]:
"""
f = tarfile.open("resources/lmd_matched.tar.gz")
f.extractall()
f.close
"""



'\nf = tarfile.open("resources/lmd_matched.tar.gz")\nf.extractall()\nf.close\n'

In [10]:
# https://github.com/craffel/pretty-midi/blob/main/Tutorial.ipynb
test_song_path = 'lmd_matched/A/A/A/TRAAAGR128F425B14B/1d9d16a9da90c090809c153754823c2b.mid'
song_pm = pretty_midi.PrettyMIDI(test_song_path)
song_pm


TypeError: the JSON object must be str, bytes or bytearray, not PrettyMIDI